In [8]:
import requests
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup as bs4
import re
import pandas as pd
import numpy as np
import json

class MOPS_ALL_2019(object):
    
    def __init__(self, sid, year, season, rid):
        self.sid=sid
        self.year=year
        self.season=season
        self.rid=rid
        keyword={'sid':self.sid, 'y':self.year, 's':self.season, 'rid':self.rid}
        self.header= {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}
        self.url='https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID={sid}&SYEAR={y}&SSEASON={s}&REPORT_ID={rid}'.format(**keyword)
        self.get_tbl()
        
    def fetchall(self):
        self.fetch_BS()
        self.fetch_PLS()
        self.fetch_CFS()
        #self.fetch_SES()
        self.jsondata = {'data': [self.Balance, self.ProfitLoss , self.CashFlows]}
        
        return self.jsondata        
    
    def get_tbl(self):
        web_ss = requests.session()
        ss_adapter = HTTPAdapter(max_retries=3)
        web_ss.mount("https://", adapter=ss_adapter)
        res = web_ss.get(url=self.url, headers=self.header, timeout=5)
        res.encoding = 'big5'
        try:
            if res.status_code == 200:
                #soup = bs4(res.text, "lxml")
                soup = bs4(res.text, "html.parser")
                self.tables = soup.select('table')
        except Exception as e:
            print(e)
            
        return self.tables
            
            
    def fetch_BS(self, Ntable=0):
        start= len(stock.tables[Ntable].select('th'))
        k=start-3          
        code = self.tables[Ntable].select('td')
        
        code_list=[]

        for _code in code:
            try:
                if (_code['style'])!='':
                    code_list.append(_code.text)
            except:
                pass
        #print(code_list)

        cht = self.tables[Ntable].select('span.zh')
        en = self.tables[Ntable].select('span.en')
        amt = self.tables[Ntable].select('td.amt')

        #print(code)
        #print(len(cht[start:]), len(amt)/k)
        #df_cht, df_eng = {} , {}

        Balance_list=[]
        j=0
        for i, _amt in enumerate(amt):

            if i%k==0:
                #print(i, _amt.text)
                #balance_list[code_list[j]]= [{'cht':cht[6+j].text.strip(),'value':_amt.text.strip().replace(',','')}]
                Balance_list.append({'code':code_list[j], 'en':en[start+j].text.strip(), 'cht':cht[start+j].text.strip()
                                      ,'value':_amt.text.strip().replace(',','')})
                j+=1
        if len(code_list)!=len(Balance_list):
            print('length is not matched--BS!!:')      
        #print(self.Balance)
        self.Balance={'Balance':Balance_list}
        return self.Balance
        
    def fetch_PLS(self, Ntable=1):
        start= len(stock.tables[Ntable].select('th'))
        k=start-3
        #print(start, k)
        code = self.tables[Ntable].select('td')
        code_list=[]

        for _code in code:
            try:
                if (_code['style'])!='':
                    code_list.append(_code.text)

            except:
                pass
        #print(code_list, len(code_list))

        cht = self.tables[Ntable].select('span.zh')
        en = self.tables[Ntable].select('span.en')
        amt = self.tables[Ntable].select('td.amt')
        

        ProfitLoss_list=[]
        j=0
        for i, _amt in enumerate(amt):
            if i%k==0:
                ProfitLoss_list.append({'code':code_list[j], 'en':en[start+j].text.strip(), 'cht':cht[start+j].text.strip(),
                                      'value':_amt.text.strip().replace(',','')})
                j+=1
                #print(i, _amt.text)
        
        if len(code_list)!=len(ProfitLoss_list):
            print('length is not matched--PLS!!:')
           
           
        self.ProfitLoss={'ProfitLoss':ProfitLoss_list}

        return self.ProfitLoss
        
    def fetch_CFS(self, Ntable = 2):
        start= len(stock.tables[Ntable].select('th'))
        k=start-3
        
        code = self.tables[Ntable].select('td')
        code_list=[]

        for _code in code:
            try:
                if (_code['style'])!='':
                    code_list.append(_code.text)
            except:
                pass
        #print(code_list)

        cht = self.tables[Ntable].select('span.zh')
        en = self.tables[Ntable].select('span.en')
        amt = self.tables[Ntable].select('td.amt')

        #print(code)

        #print(len(cht[start:]), len(amt)/2)

        CashFlows_list=[]
        j=0
        for i, _amt in enumerate(amt):

            if i%k==0:
                CashFlows_list.append({'code':code_list[j], 'en':en[start+j].text.strip(), 'cht':cht[start+j].text.strip(),
                                      'value':_amt.text.strip().replace(',','')})
                j+=1
        
           
        if len(code_list)!=len(CashFlows_list):
            print('length is not matched--CFS!!:') 
        self.CashFlows={'CashFlows':CashFlows_list}  
        
        
        return self.CashFlows
        
        
    def fetch_SES(self, Ntable = 3):
        #start= len(stock.tables[Ntable].select('th'))
        #k=start-3

        Etable = self.tables[Ntable]
        #print(Etable)

        td=Etable.select('td')

        CI=[]
        for _td in td:
            if re.findall('^[A-Z][0-9]', _td.text.strip()) !=[]:
                CI.append(re.findall('^[A-Z][0-9]', _td.text.strip()))
                #print(code_index[0])


        th=Etable.select('th')
        cht = Etable.select('span.zh')
        en = Etable.select('span.en')
        amt = Etable.select('td.amt')

        #code4=[]
        self.code4={}

        j=1
        for _th in th:
            if re.findall('^3.+', _th.text.strip()) !=[]:
                self.code4[_th.text.strip()]={'cht': cht[j].text.strip(),'en':en[j].text.strip()}
                j+=1
        #print(code4)
        #print(code4.keys())

        #index=[]
        self.index2={}
        for _cht, _en, _CI in zip(cht[j:], en[j:], CI):
            #index.append({'code_index':_CI[0],'cht_index':_cht.text.strip(), 'en_index': _en.text.strip()})
            self.index2[_CI[0]]={'cht': _cht.text.strip(),'en': _en.text.strip()}
            #print(code_index)
            #print(cht_index)
            #print(en_index)               
        #print(index2)
        #print(index2.keys())

        k1= len(self.code4)
        k2= len(self.index2.keys())
 
        #for _amt in amt:
        #    print(_amt.text)
        amt_row= [[] for x in range(k2)]

        j=0
        for i, _amt in enumerate(amt):
            #print(i, _amt.text)
            if i==0 or i % k1!= 0:
                amt_row[j].append(_amt.text.strip().replace(',','').replace('(','-').replace(')',''))
                #print(amt_row[j])
            else:
                j+=1
                if j<k2:
                    amt_row[j].append(_amt.text.strip().replace(',','').replace('(','-').replace(')',''))
                else:
                    break
                    
        #print(amt_row)

        self.df=pd.DataFrame(columns=self.code4.keys())
        self.df=pd.DataFrame(np.array(amt_row),columns=self.code4.keys())
        self.df['items']=self.index2.keys()
        self.df.set_index('items', inplace=True) 
        self.Account={'code4':self.code4, 'index2':self.index2}
        return self.df

# Fetch DATA

In [ ]:
import os
import time
import random
from requests.exceptions import ConnectionError
from codes import codes


def dlcheck(path):
    filename=os.listdir(path)
    code_cap=[]
    for _file in filename:
        code_cap.append(_file[0:4])
    return code_cap

year='2020'
season='1'
path=year+'Q'+season
csvpath=year+'Q'+season+'SES'

if not os.path.exists(path):
    os.mkdir(path)

if not os.path.exists(csvpath):
    os.mkdir(csvpath)

#取得兩個資料夾的交集，以便確認是否抓過
code_cap=set(dlcheck(path)) & set(dlcheck(csvpath))
#print(code_cap)


for code,v in codes.items():
    if v.type=="股票" and v.market=="上市":
        try:
            if code not in code_cap:
                rid='C'           
                stock=MOPS_ALL_2019(code,year,season,rid)

                if stock.tables == []:
                    rid = 'A'
                    stock=MOPS_ALL_2019(code,year,season,rid)
                             
                if stock.tables != []:
                    
                    print('fetch BS, PLS and CFS from:' ,stock.url)
                    data = stock.fetchall()
                    filename=path+'/'+code+'-'+year+'-'+'Q'+season+'.json'
                    with open(filename, 'w', encoding='utf8') as f:
                        json.dump(data, f)
                    
                    
                    print('fetch SES from:' ,stock.url)
                    df = stock.fetch_SES()
                    csvname=csvpath+'/'+code+'-'+year+'-'+'Q'+season+'.csv'
                    df.to_csv(csvname)
                    jsonname=csvpath+'/'+code+'-'+year+'-'+'Q'+season+'.json'
                    with open(jsonname, 'w', encoding='utf8') as f:
                        json.dump(stock.Account, f)
                        
                        
                else:
                    print('error-nodata:', code) 
                    with open('error.log', 'a+') as f:   
                        f.write('nodata:'+ code+':' + stock.url+'\n')
                time.sleep(random.uniform(2,5))             
                
        except ConnectionError:
            code_cap=dlcheck(path)
            time.sleep(120)
            continue
print('download finished!')

{'2832', '2462', '2607', '2390', '2380', '1702', '2528', '1714', '1234', '2496', '1235', '1449', '2851', '2375', '1605', '2476', '2303', '1301', '1476', '1723', '1711', '1795', '2376', '2002', '2472', '1103', '2939', '2491', '2723', '1307', '2633', '2484', '1445', '2936', '1717', '2327', '1507', '2347', '2459', '1213', '1441', '1773', '2020', '1102', '1513', '2105', '2393', '2613', '2412', '2354', '2809', '2227', '2430', '1312', '2616', '2363', '2617', '2314', '2890', '1734', '2207', '1612', '2426', '2701', '1474', '2033', '2415', '1535', '2884', '2392', '1909', '2465', '1905', '1506', '2850', '1760', '2480', '1434', '2836', '2342', '2509', '1323', '1459', '1537', '1589', '2201', '2349', '2614', '2615', '2605', '1475', '1762', '2408', '1733', '1409', '2233', '2427', '2377', '2461', '2466', '2024', '1477', '2104', '2387', '1439', '2439', '1454', '3005', '2483', '2009', '1416', '1583', '2535', '2364', '2855', '3006', '2014', '2324', '2007', '2069', '2062', '1219', '1310', '2929', '2889',

fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=3032&SYEAR=2020&SSEASON=1&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=3032&SYEAR=2020&SSEASON=1&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=3033&SYEAR=2020&SSEASON=1&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=3033&SYEAR=2020&SSEASON=1&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=3034&SYEAR=2020&SSEASON=1&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=3034&SYEAR=2020&SSEASON=1&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=3035&SYEAR=2020&SSEASON=1&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=3035&SYEAR=2020&SSEASON=1&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.co

# Check ALL

In [ ]:
#path='AllRpt'
year='2019'
season='4'
rid='C'
code='1101'

#path='AllRpt'
path=year+'Q'+season

if not os.path.exists(path):
    os.mkdir(path)
code_cap=dlcheck(path)


stock=MOPS_ALL_2019(code,year,season,rid)
filename=path+'/'+code+'-'+year+'-'+'Q'+season+'.json'
print('get data from:' ,stock.url)
data = stock.fetchall()
with open(filename, 'w', encoding='utf8') as f:
    json.dump(data, f)

#stock.tables[1]

data=json.load(open(filename))['data']
#print(data[1]['ProfitLoss'])
df=pd.DataFrame(data[1]['ProfitLoss'])
df

# Equity Check

In [5]:
import os

year='2019'
season='3'
rid='C'
code='1101'

csvpath=year+'Q'+season+'SES'

if not os.path.exists(csvpath):
    os.mkdir(csvpath)

stock=MOPS_ALL_2019(code,year,season,rid)

csvname=csvpath+'/'+code+'-'+year+'-'+'Q'+season+'.csv'
print('get data from:' ,stock.url)


df = stock.fetch_SES()
df.to_csv(csvname)


jsonname=csvpath+'/'+code+'-'+year+'-'+'Q'+season+'.json'
with open(jsonname, 'w', encoding='utf8') as f:
    json.dump(stock.Account, f)

df

get data from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1101&SYEAR=2019&SSEASON=3&REPORT_ID=C


,3110,3120,3130,3140,3150,3190,3100,3200,3310,3320,...,3300,3410,3420,3425,3450,3400,3500,31XX,36XX,3XXX
items,,,,,,,,,,,,,,,,,,,,,
A1,51080599,2000000,,,,,53080599,47836241,14784534,13049062,...,61588761,-5037221,24074566,0,1109,19038454,-2545,181541510,15837946,197379456
A3,0,0,,,,,0,,0,0,...,0,,,,,,,0,0,0
A4,0,0,,,,,0,,0,0,...,0,,,,,,,0,0,0
A5,51080599,2000000,,,,,53080599,47836241,14784534,13049062,...,61588761,-5037221,24074566,0,1109,19038454,-2545,181541510,15837946,197379456
B1,,,,,,,,,2118082,,...,0,,,,,,,0,,0
B3,,,,,,,,,,,...,,,,,,,,0,,0
B5,,,,,,,,,,,...,-16856367,,,,,,,-16856367,,-16856367
B7,,,,,,,,,,,...,-18219,,,,,,,-18219,,-18219
B9,3575593,,,,,,3575593,,,,...,-3575593,,,,,,,0,,0
